In [7]:
import xgboost as xgb
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import VGG16_Weights

# Load the pre-trained XGBoost model
xgb_model =  xgb.XGBRegressor()
xgb_model.load_model("xgb_model.json")

# Load the pretrained VGG16 model. Setting `pretrained=True` loads weights trained on ImageNet.
model = models.vgg16(weights=VGG16_Weights.DEFAULT)
# We only need the features, so we remove the classifier part by taking only `model.features`.
model = model.features
# Set the model to evaluation mode to prevent training-related behavior, such as dropout.
model.eval()

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # VGG16 normalization
                         std=[0.229, 0.224, 0.225])
])

# Function to extract features from the image
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return np.zeros(4096)  # Return a zero vector if image is not found or corrupted
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(image)
    return features.numpy().flatten().reshape(1, -1)

# Prepare the text features
def preprocess_text_features(features):
    return np.array(features).reshape(1, -1)  # Ensure it is a 2D array

In [20]:
# Combine image and text features
image_path = "data/socal_pics/1.jpg"
image_features = extract_features(image_path)
text_features = preprocess_text_features([1560,48, 5, 3.0])

# Concatenate all features
all_features = np.hstack((text_features, image_features))

In [21]:
# Predict with the XGBoost model
# dtest = xgb.DMatrix(all_features.reshape(1, -1))
predicted_price = xgb_model.predict(all_features)

print("Predicted Price:", predicted_price[0])

Predicted Price: 569464.3
